In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pickle

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [41]:
#scoreType = 'LIX'
scoreType = 'SumOfScores'
#scoreType = 'Difficulty'
#scoreType = 'Complexity'

subj_session_name = 'sa\\Test_woChinRest'

subj_session_name_fig = subj_session_name
#subj_session_name_fig = subj_session_name + '_p1'
#subj_session_name_fig = subj_session_name + '_p2'


In [27]:
# open the interactive figures 


figx = pickle.load(open(r'C:\DTU\Results\201812_PilotExptForMovement\pupil Size\Complete pupil size\rh_Test_woChinRest', 'rb'))

figx.show() # Show the figure, edit it, etc.!

In [43]:
absoluteSize = True # True means absolute, False means relative

In [29]:
def OptiKeyTypingTime(userKeys):
    
    timeTyping = dict()
    
    time1, t1, t2 = userKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = userKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    timeTyping['startTime'] = startTime
    timeTyping['endTime'] = endTime
    
    return timeTyping

In [10]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [11]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [12]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    

    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [13]:
def pupilSize_CompleteSignal(GazeLog, timeTyping, pupilData, subjName):
    
    pupilData['RLCorrelation'] = []
    pupilLogL = list()
    pupilLogR = list()

    # create a list of time of gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # Create list of pupil sizes from gazelog
#     pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in GazeLog]
#     pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in GazeLog]
    
    pupilLogL_beforeDecimal = [item4[-5] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    pupilLogL_afterDecimal = [item4[-4] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    pupilLogR_beforeDecimal = [item4[-2] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    pupilLogR_afterDecimal = [item4[-1] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
    for i in range(0, len(pupilLogL_beforeDecimal)):
        #print(type(pupilLogL_beforeDecimal[i]), pupilLogL_beforeDecimal[i], type(pupilLogL_afterDecimal[i]), pupilLogL_afterDecimal[i])
        if 'Valid' not in pupilLogL_beforeDecimal[i] and 'Valid' not in pupilLogL_afterDecimal[i]:
            if 'nan' not in pupilLogL_beforeDecimal[i] and 'nan' not in pupilLogL_afterDecimal[i]:
                #print(type(pupilLogL_beforeDecimal[i]), pupilLogL_beforeDecimal[i], type(pupilLogL_afterDecimal[i]), pupilLogL_afterDecimal[i])
                pupilLogL.append(float(pupilLogL_beforeDecimal[i]+'.'+pupilLogL_afterDecimal[i]))
            else:
                pupilLogL.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            pupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
    
    for i in range(0, len(pupilLogR_beforeDecimal)):
        if 'Valid' not in pupilLogR_beforeDecimal[i] and 'Valid' not in pupilLogR_afterDecimal[i]:
            if 'nan' not in pupilLogR_beforeDecimal[i] and 'nan' not in pupilLogR_afterDecimal[i]:
                #print(type(pupilLogL_beforeDecimal[i]), pupilLogL_beforeDecimal[i], type(pupilLogL_afterDecimal[i]), pupilLogL_afterDecimal[i])
                pupilLogR.append(float(pupilLogR_beforeDecimal[i]+'.'+pupilLogR_afterDecimal[i]))
            else:
                pupilLogR.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            pupilLogR.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(timeGazeLog, timeTyping['startTime'])
    timeEnd, timeEndInd = nearestTimePoint(timeGazeLog, timeTyping['endTime'])
        
    typingPupilL = pupilLogL[timeStartInd:timeEndInd]
    typingPupilR = pupilLogR[timeStartInd:timeEndInd]
    
    # find internal start and end time
    timeInternalRaw = timeInternalGazeLog[timeStartInd:timeEndInd] 
    
    # find difference in consecutive elements of internal time
    timeInternalDifference = [t - s for s, t in zip(timeInternalRaw, timeInternalRaw[1:])]
 
    # divide by 1000 to make it s
    timeOfGazeLog = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

    # filter gaze data using hampel filter and compute average
    
    winSize = 25
    # Filter pupil sizes
    pupilHampelFilteredL = hampel(typingPupilL, winSize, 3)
    pupilHampelFilteredR = hampel(typingPupilR, winSize, 3)
    
    # compute pupil correlation and go on, only if correlation is greater than 0.5
    pupilCorr = pupilHampelFilteredL.corrwith(pupilHampelFilteredR, axis = 0)
    
    pupilData['RLCorrelation'].append(pupilCorr.values[0])

    print(pupilData['RLCorrelation'], 'from: ', timeStart, 'to: ', timeEnd)
    # 0.8 is a good value for correlation and was also the mean of mean of the correlations
    # for the trials for the users
    
    # Moving Mean of data without outliers: 
    pupilAbsoluteL = pupilHampelFilteredL.rolling(window=winSize, min_periods=1, center=True).mean()
    pupilAbsoluteR = pupilHampelFilteredR.rolling(window=winSize, min_periods=1, center=True).mean() 
            
    pupilRelativeL = (pupilAbsoluteL - np.mean(pupilAbsoluteL[0][0:22]))/np.mean(pupilAbsoluteL[0][0:22])
    pupilRelativeR = (pupilAbsoluteR - np.mean(pupilAbsoluteR[0][0:22]))/np.mean(pupilAbsoluteR[0][0:22])
    
    
    
    if absoluteSize: 
        pupilData['LeftFiltered'] = pupilAbsoluteL
        pupilData['RightFiltered'] = pupilAbsoluteR
    else:  
        pupilData['LeftFiltered'] = pupilRelativeL
        pupilData['RightFiltered'] = pupilRelativeR
    
    pupilData['LeftRaw'] = typingPupilL
    pupilData['RightRaw'] = typingPupilR    
    
      
    
    
    return pupilData, timeOfGazeLog



In [22]:
def plotCompletePupilSize(pupilData, timeData, subjName):
    
    fig = plt.figure()
    axL = fig.add_subplot(2,1,1)
    axL.plot(timeData, pupilData['LeftRaw'][0:len(timeData)], 'b', label = 'Before blink removal')
    axL.plot(timeData, pupilData['LeftFiltered'][0:len(timeData)], 'r', label = 'After blink removal')
    plt.legend()
    
    axR = fig.add_subplot(2,1,2)
    axR.plot(timeData, pupilData['RightRaw'][0:len(timeData)], 'b', label = 'Before blink removal')
    axR.plot(timeData, pupilData['RightFiltered'][0:len(timeData)], 'r', label = 'After blink removal')
    plt.legend()

    axL.set_title('Left')
    #axL.set_xlabel('Time [in s]')
    axR.set_title('Right')
    axR.set_xlabel('Time [in s]')

    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
        axL.set_ylabel('Absolute pupil size [in mm]')
        absoluteOrRelative = '_absolute'
    else:
        axR.set_ylabel('Relative pupil size [in %]')
        axL.set_ylabel('Relative pupil size [in %]')
        absoluteOrRelative = '_relative'

    axL.set_title(subj_session_name)
    plt.rcParams.update({'font.size': 20})
    
    
    pickle.dump(fig, open(r'C:\DTU\Results\201812_PilotExptForMovement\pupil Size\Complete pupil size' + '\\' + subj_session_name_fig + absoluteOrRelative, 'wb'))


In [44]:
subjName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []

for root, dirs, subfolder in os.walk(subjName):
    if not dirs and subj_session_name in root:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or gazeLog is None:
                continue
            else:
                
                pupilData = dict()
                
                # call function to check when first key is looked at, and when quit key is selected
                timeTyping = OptiKeyTypingTime(userKeys)
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                if 'p1' in root:
                    subj_session_name_fig = subj_session_name + '_p1'
                elif 'p2' in root:
                    subj_session_name_fig = subj_session_name + '_p2'
                else:
                    subj_session_name_fig = subj_session_name
                    
                pupilData, timeData = pupilSize_CompleteSignal(gazeLog, timeTyping, pupilData, subjName)
                
                plotCompletePupilSize(pupilData, timeData, subjName)

sa\Test_woChinRest\p1
[0.9011047788416573] from:  2018-12-04 13:30:45.202837 to:  2018-12-04 13:50:39.471128
sa\Test_woChinRest\p2
[0.8863348145415606] from:  2018-12-05 11:08:27.023053 to:  2018-12-05 11:35:26.638887


In [12]:
print(len(pupilData['LeftFiltered']))

202830


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(timeData, pupilData['LeftRaw'][0:len(timeData)], 'b')
ax.plot(timeData, pupilData['RightRaw'][0:len(timeData)], 'r')
ax.legend('Before blink removal', 'After blink removal')
    
ax.set_title('Left')
ax.set_xlabel('Time [in s]')
if absoluteSize:
    ax.set_ylabel('Absolute pupil size [in mm]')
else:
    ax.set_ylabel('Relative pupil size [in %]')
    
ax.set_title(subjName)
plt.rcParams.update({'font.size': 22})

In [ ]:
a = [2, 4, 1]
b = [4, 6, 3]

c = [float(str(a[i])+'.'+str(b[i])) for i in range(0, len(a))]

In [ ]:
c